# Creating simulated data from a mosaic image

This notebook demonstrates how to use Mirage to create simulated data from a distortion-free mosaic image. In this case, we will use a mosaic of the GOODS-S region from the CANDELS survey. (****ADD LINKS for candels and goodss****)

For each observation to be simulated, the appropriate area of the mosaic is extracted from the full mosaic, and is resampled in order to introduce the distortion associated with the JWST instrument to be used. This distorted image is then addded to the simulated data in one of two ways.

If you wish to modify the image from the mosaic in any way, such as adding additional objects or scaling the brightness, then the mosaic image can be added to one of Mirage's "extended" source catalogs, along with additional sources.

If you do not wish to modify the cropped mosaic image in any way (other than introducing the appropriate distortion), then the distorted image can be used directly as a seed image, and you only need to run the dark_prep and obs_generation steps of Mirage in order to create the final simulated data.

In [ ]:
import os
import yaml

In [ ]:
from mirage.catalogs.catalog_generator import ExtendedCatalog
from mirage.catalogs.create_catalog import combine_catalogs
from mirage.dark.dark_prep import DarkPrep
from mirage.ramp_generator.obs_generator import Observation
from mirage.imaging_simulator import ImgSim
from mirage.seed_image.fits_seed_image import ImgSeed
from mirage.yaml import yaml_generator

## Download data

In [ ]:
base_url = ('https://data.science.stsci.edu/redirect/JWST/jwst-simulations/mirage_reference_files'
            '/example_data_for_notebooks/')

In [ ]:
input_images = ['3C305.1_stamp.fits', '3C324_stamp.fits', '3C454.1_stamp.fits',
                'hlsp_candels_hst_wfc3_gs-tot-sect23_f160w_v1.0_drz']

In [ ]:
for filename in input_images:
    full_url = '{}{}'.format(base_url, filename)
    download here

## Using blotted image in an extended source catalog

In [ ]:
mosaicfile = 'gs_all_candels_ers_f160w_060mas_v0.5_drz.fits'
xml_file = 'extended_object_test.xml'
pointing_file = xml_file.replace('.xml', '.pointing')

User-inputs to the yaml generator. Note that you can still use a catalogs input here and add
point sources or galaxies. Extended source catalog names will be added later

In [ ]:
cr = {'library': 'SUNMAX', 'scale': 1.0}
dates = '2019-5-25'
background = 'low'
pav3 = 0.0
#catalogs = Show example with point source / galaxy, but comment out

Run the yaml generator

In [ ]:
yam = yaml_generator.SimInput(xml_file, pointing_file, verbose=True,
                              output_dir='yamls',
                              cosmic_rays=cr,
                              #catalogs=catalogs,
                              background=background, roll_angle=pav3, dates=dates,
                              simdata_output_dir='simdata',
                              datatype='raw')
yam.use_linearized_darks = True
yam.create_inputs()

Get a list of all instruments, apertures, and filters used in the APT file

In [ ]:
instruments = yam.info['Instrument']
filter_keywords = ['FilterWheel', 'ShortFilter', 'LongFilter', 'Filter']
pupil_keywords = ['PupilWheel', 'ShortPupil', 'LongPupil']
yam.info

nrc_sw_optics = set([(f, p) for f, p in zip(yam.info['ShortFilter'], yam.info['ShortPupil'])])
nrc_lw_optics = set([(f, p) for f, p in zip(yam.info['LongFilter'], yam.info['LongPupil'])])
niriss_optics = set([(f, p) for f, p in zip(yam.info['FilterWheel'], yam.info['PupilWheel'])])
niriss_wfss_optics = set([(f, p) for f, p in zip(yam.info['Filter'], yam.info['PupilWheel'])])

print('NIRCam filters/pupils used in this proposal: ')
print(nrc_sw_optics)
print(nrc_lw_optics)
print('\nNIRISS filters/pupils used in this proposal: ')
print(niriss_optics)
print(niriss_wfss_optics)
print(('\nBe sure to add magnitude columns to the template catalog '
        'for all filters you are going to simulate.\n'))

Create a template extended source catalog containing sources other than the mosaic image that you want to add to the seed image. The resampled mosaic will be added to this template later. Note that you must add magnitude values for these other sources in all filters that are used in the proposal.

In [ ]:
filter1 = 'F150W'
filter2 = 'F444W'

In [ ]:
other_stamp_files = ['n49_0.5-7.0_flux.fits', 'n49_0.5-7.0_flux.fits', 'n49_0.5-7.0_flux.fits']
other_stamp_ra = [53.126751, 53.122751, 53.122751]
other_stamp_dec = [ -27.801089, -27.801089, -27.809089]
other_stamp_pa = [0., 0., 0.]
other_stamp_f150w_mags = [16., 17., 17.5]
other_stamp_f444w_mags = [15., 16., 16.5]

# Magnitude values must be strings here because we will be combining them
# with values of 'None' for the resampled image magnitudes
f150w_mags_as_str = [str(element) for element in other_stamp_f150w_mags]
f444w_mags_as_str = [str(element) for element in other_stamp_f444w_mags]

template_extended_catalog_file = 'extended_sources_template.cat'
template_cat = ExtendedCatalog(filenames=other_stamp_files, ra=other_stamp_ra, dec=other_stamp_dec,
                               position_angle=other_stamp_pa)
template_cat.add_magnitude_column(f150w_mags_as_str, instrument='nircam', filter_name=filter1)
template_cat.add_magnitude_column(f444w_mags_as_str, instrument='nircam', filter_name=filter2)
template_cat.save(template_extended_catalog_file)

In [ ]:
# Directory in which to save the resampled images
resampled_output_dir = './'

For each yaml file, create a resampled image from the mosaic, add this image to the template catalog and save as a yaml-specific catalog, and enter the catalog name in the yaml file

In [ ]:
for yfile in yam.yaml_files:
    
    # Read in the yaml file so that we know RA, Dec, PAV3
    # of the exposure
    with open(yfile) as file_obj:
        params = yaml.safe_load(file_obj)
        
    ra = params['Telescope']['ra']
    dec = params['Telescope']['dec']
    pav3 = params['Telescope']['rotation']

    # Define the output files and directories
    sim_data_dir = params['Output']['directory']
    simulated_filename = params['Output']['file']
    crop_file = simulated_filename.replace('.fits', '_cropped_from_mosaic.fits')
    crop_file = os.path.join(sim_data_dir, crop_file)
    blot_file = simulated_filename.replace('.fits', '_blotted_seed_image.fits')
    
    # Crop from the mosaic and resample for the desired detector/aperture
    seed = ImgSeed(paramfile=yfile, mosaicfile=mosaicfile, cropped_file=crop_file,
                   outdir=sim_data_dir, blotted_file=blot_file)
    seed.crop_and_blot()

    # Now add the resampled file to the extended source catalog template and
    # save as a separate catalog file
    
    # Need to add a magnitude entry for each filter/pupil
    mosaic_f150w_mag = ['None']
    mosaic_f444w_mag = ['None']
    
    # Create the catalog containing only the resampled image
    blotted_image_full_path = os.path.join(sim_data_dir, blot_file)
    extended_catalog_file = simulated_filename.replace('.fits', '_extended_sources.cat')
    ext_cat = ExtendedCatalog(filenames=[blotted_image_full_path], ra=[ra], dec=[dec], position_angle=[pav3])
    ext_cat.add_magnitude_column(mosaic_f150w_mag, instrument='nircam', filter_name=filter1)
    ext_cat.add_magnitude_column(mosaic_f444w_mag, instrument='nircam', filter_name=filter2)

    # Combine the resampled image catalog and the template catalog
    combined_cat = combine_catalogs(ext_cat, template_cat)
    combined_cat.save(extended_catalog_file)

    # Now add this extended source catalog to the yaml file
    params['simSignals']['extended'] = extended_catalog_file

    # Save the updated yaml file
    with open(yfile, 'w') as file_obj:
        dump = yaml.dump(params, default_flow_style=False)
        file_obj.write(dump)

Create the simulated data

In [ ]:
for yfile in yam.yaml_files[0:1]:
    sim = ImgSim(paramfile=yfile)
    sim.create()

## Use resampled image as a seed image

In [ ]:
mosaicfile = 'gs_all_candels_ers_f160w_060mas_v0.5_drz.fits'
xml_file = 'extended_object_test.xml'
pointing_file = xml_file.replace('.xml', '.pointing')

User-inputs to the yaml generator. Note that you can still use a catalogs input here and add
point sources or galaxies. Extended source catalog names will be added later

In [ ]:
cr = {'library': 'SUNMAX', 'scale': 1.0}
dates = '2019-5-25'
background = 'low'
pav3 = 0.0
#catalogs = Show example with point source / galaxy, but comment out

Run the yaml generator

In [ ]:
yam = yaml_generator.SimInput(xml_file, pointing_file, verbose=True,
                              output_dir='yamls',
                              cosmic_rays=cr,
                              #catalogs=catalogs,
                              background=background, roll_angle=pav3, dates=dates,
                              simdata_output_dir='simdata',
                              datatype='raw')
yam.use_linearized_darks = True
yam.create_inputs()

For each yaml file, create a resampled image from the mosaic, add this image to the template catalog and save as a yaml-specific catalog, and enter the catalog name in the yaml file

In [ ]:
for yfile in yam.yaml_files:
    
    # Read in the yaml file so that we know RA, Dec, PAV3
    with open(yfile) as file_obj:
        params = yaml.safe_load(file_obj)
        
    # Define output filenames and directories
    sim_data_dir = params['Output']['directory']
    simulated_filename = params['Output']['file']
    crop_file = simulated_filename.replace('.fits', '_cropped_from_mosaic.fits')
    crop_file = os.path.join(sim_data_dir, crop_file)
    blot_file = simulated_filename.replace('.fits', '_blotted_seed_image.fits')
        
    # Crop from the mosaic and then resample the image
    seed = ImgSeed(paramfile=yfile, mosaicfile=mosaicfile, cropped_file=crop_file,
                   outdir=sim_data_dir, blotted_file=blot_file)
    seed.crop_and_blot()
    
    # Run dark_prep
    dark = DarkPrep()
    dark.paramfile = yfile
    dark.prepare()

    # Run the observation generator
    obs = Observation()
    obs.paramfile = yfile    
    obs.seed = seed.seed_image
    obs.segmap = seed.seed_segmap
    obs.seedheader = seed.seedinfo
    obs.linDark = dark.prepDark
    obs.create()

Now run the dark_prep and obj_generator modules

In [ ]:
This needs to be done in the same loop as the cropping and blotting
if we are going to pass the seed_image, segmap, and seedinfo to obs_generator.
If we pass a file then this can be in a separate loop.
for yfile in yam.yaml_files:
    # Run dark_prep
    dark = DarkPrep()
    dark.paramfile = yfile
    dark.prepare()

    # Run the observation generator
    obs = Observation()
    obs.paramfile = yfile

    # This, or
    #obs.seed = seed.blotted_file
    # This
    obs.seed = seed.seed_image
    obs.segmap = seed.seed_segmap
    obs.seedheader = seed.seedinfo

    obs.create()

In [ ]:
seed.seed_image